# Intro to curve fitting in Python for Chemists

This lesson aims to teach how to fit data in Python using the `SciPy` package. It is recommended that you have completed the [intro to Python Colab](https://colab.research.google.com/drive/1ltdsUiprT3O1kZIhEXF69ZJhHlqRPApx?usp=sharing) before attempting this one. The [data visualization Colab](https://colab.research.google.com/drive/1oWQn7g9zPEF605geLLMzfxOH7YOGH531?usp=sharing) may be helpful here as well but is not required.

A few things to start:

1.   These lessons only work in Google Chrome
2.   If you want to save your progress, go to File> Save a Copy in Drive; then locate a spot in your Drive folder
3.   You can save these notebooks and run offline as a Jupyter Notebook.

If you have questions, feel free to contact Dr. Chris Berndsen in the JMU Chemistry Department.



---



# Load the Python packages
Push the play button to load the python packages taking special note of how each package can be called by looking after the word "as".

In [ ]:
# import the packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit


The data set we are going to import is from a set enzyme assays taken at several concentrations of the substrate. The data are absorbance measurements over time, which can be used to calculate the reaction rate at each substrate concentration. Ultimately, the data can be fitted to determine the Vmax and Kₘ value for the enzyme. There are several tasks to complete:



1.   Plot the data to determine the appropriate time range to calculate the initial rate
2.   Calculate the initial rate for each substrate concentration
3.   Fit the initial rate data to the Michaelis-Menten equation
4.   Plot the initial rate data and the fit

There are several ways to approach these tasks and you are free to try a number of ways within the steps below. We expect that to complete these tasks you will likely use the `matplotlib`, `numpy`, `pandas`, and `scipy` packages and a `for` loop.



In [ ]:
# import the data from github
url = 'https://raw.githubusercontent.com/CEBerndsen/chem_colab_data/main/ldh_kinetics_data.csv'
df = pd.read_csv(url)

# peek at the organization of the data
# how many rows
print("The data has", len(df), "rows")



# top 6 rows and columns
df.head()



---

# Viewing and cleaning the data




Now that the data are imported, it is time to understand how the data can be fitted correctly. In enzyme kinetics, scientists use the initial rate, where the rate of the substrate production is linear with time. This is called the steady-state rate.

Using `matplotlib`, make a plot of the data at the highest substrate concentration, the lowest substrate concentration, and one of the middle concentrations. The basic framework for the plot is in the box below, fill in the missing parts.

In [ ]:
# plot the data
plt.plot(df['time'], ________, 'ro')
plt.plot(__________, ________, 'bo')
plt.plot(__________, ________, 'go')
plt.show()

Based on the plots above, it appears that product formation is linear with time until 60-100 seconds in. At the highest concentration, the data may not fully include the the steady-state, because the enzyme was catalyzing the reaction very quickly. We may remove this subset of the data later but will keep it in until we are certain.

A good practice would be to view the data from substrate concentrations in the upper half of the range to know where the dataset begins to lose information on the enzyme steady-state.


---

## Filtering the data

To remove the data that occur after a certain time we need to indicate which column we will use and the condition we will use to indicate what rows to remove. In this case we want the `time` column and we want to keep the rows where time is < than the cutoff. We are making a subset of the original data where the data are linear.

Fill in the blanks in the code below and then plot the data as above to confirm the data were removed correctly.




In [ ]:
# fill in the blanks
# blank one is the column name
# blank two is a condition operator either >, <, or ==
# blank three is the time value
df2 = df[df['_______'] __ _____]

# plot the data as above to confirm the filtering was done correctly.
# remember to change the dataframe name if you just copy/paste the code from above!



plt.show()

If the filtering worked correctly, then the x axis should run from 0 to your cutoff value and each data set should be roughly linear. If either or both of those are not true, then check your code and try again.


---

# Calculating the initial rates with a `for` loop

Now that the data are reduced to just the data for calculating the inital rate, it is time to fit each data to a linear equation to determine the slope, which is equal to the rate. First we need to create the model or equation that we will fit to the data.

The `SciPy` package contains a function called `curve_fit` which takes an equation, an x-values dataset, and a y-values dataset to determine the best fit. In the next code block, the equation is defined.





In [ ]:
# define the function or equation for curve_fit
def lm_fit(time, m, b):
    rate = m*time + b
    return rate

Next we need to isolate the time values as an array. This makes the 'for' loop we will construct in a moment a bit easier to write.

Then create a list from the column values which are the concentrations. This will help later on when we create the dataframe with the results.

In [ ]:
# create a list from the time column
conc = list(df2.columns.values)

# create an array of concentration
timearr = df2['time'].values

Now, we will create the `for` loop. This loop will iterate through the columns and fit the data to the equation `lm_fit` that was defined above. Curve_fit will produce the rate value (popt) and the covariance (pcov). The latter of which we will convert to the standard error. Then we will append the data to the respective empty lists.

In [ ]:
# create empty lists to put the results
rate = []
error = []

# loop through the columns and fit the data
for i in ___.columns: # fill in the dataframe name
    popt, pcov = curve_fit(lm_fit, _________, df2[i]) # fill in the column to use as x using df['____'], replace ____ with the column name
    p_sigma = np.sqrt(np.diag(pcov)) # convert pcov to standard error
    rate.append(popt[0]) # add the rate to the list
    error.append(p_sigma[0]) # add the standard error to the list

# assemble everything into a nice dataframe
result_df = pd.DataFrame({'conc': conc,
                          'rate': rate,
                          'error': error})

# view the results
result_df.head()

The result should show a dataframe with three columns, but something is wrong. The first concentration is time and the rate is just the correlation of time with time.

To remove this row, we could filter the rows as we did before and remove everything with a rate of 1 or we can simply cut out the first row. The latter option is probably cleaner since we cannot be certain that there are not correct rows with a rate of 1.  
The code for both methods is shown in the next block.


In [ ]:
# remove those rows with rates equal to 1
mm2 = result_df[result_df['rate'] != 1] # keep rows where the rate != (not equal) to 1
mm2.head()

# cut out the row
mm = result_df.iloc[1:,:] # remember python starts with 0, we said keep row 1 to the end and all columns with [1:,:]
mm.head()

Now let's plot the data

In [ ]:
# fill in the column names from the mm to see how the rate varies with each concentration
plt.plot(mm['_____'], mm['_____'], 'ro')
plt.show()

Uh oh, this doesn't look correct. The x-axis labels are spaced as if the concentrations are text and not numbers. Check the type of data for each column. Ideally everything is an integer or a float, which means a number.

In [ ]:
mm.dtypes

The issue is when we did the for loop the time in the concentration column confused Python, so it made everything a character. Therefore, we need to convert the `conc` column to a number.

In [ ]:
mm['conc'] = pd.to_numeric(mm['conc'])
mm.dtypes

Everything seems better now. Next we need to convert the rates from abs/sec to micromolar (uM) per second using Beer's law. We will also need to convert the error column from absorbance to uM using the same equation. The extinction coefficient is 6220 M-1 cm-1 and the pathlength is 0.2 cm. Then multiply by 1e6 to convert M to uM.

Fill in the blanks to perform the conversion.

In [ ]:
# create a new column rate2 and error2 from the Beer's law coversion of rate and error
mm['rate2'] = mm['_____'] # add in the Beer's law equation to convert
mm['error2'] = mm['_____'] # add in the Beer's law equation to convert
mm.head()

Plot the data with rate2 on y-axis and conc on the x-axis to view the effect of substate concentration on the enzyme rate.

In [ ]:
# plot the data


plt.show()

Now we will fit the data to the Michaelis-Menten (M-M) equation by defining our equation and using curve_fit above. The M-M equation is



```
# rate = (vmax*conc)/(k_m + conc)
```
Where vmax is the maximum rate of the enzyme, k_m is the concentration of substrate that results in 1/2 Vmax, and conc is the concentration of substrate. The data we have contains conc and the rate, thus we can fit this equation or model to the  data to calculate vmax and k_m.



---


The basic code is outlined below to help you. This time it is a bit easier since we do not need to use curve_fit and a for loop, just curve_fit.

In [ ]:
# make the model for fitting
def mm_fit(conc, v_max, k_m):
    return (v_max*conc)/(k_m + conc) # we do not need to specify rate here

# review curve fit from above, what values does it produce that we can put in the blanks?
_______, ________ = curve_fit(mm_fit, ________, mm['rate'], # fill in the x data we will use
                       sigma = mm['error'], # weight the data using the error column
                       absolute_sigma = True) # it is absolute error


print("The Vmax value is", popt[0])
print("The Km value is", popt[1])

And now plot the data with the fit:


In [ ]:
plt.plot(mm['conc'], mm['rate'], 'ro')
plt.plot(mm['conc'], mm_fit(mm['conc'], *popt), 'g--',
        label = 'fit: Vmax = %5.3f, Km = %5.3f' % tuple(popt))
plt.errorbar(mm['conc'], mm['rate'], yerr = mm['error'], fmt = 'none')
plt.xlabel('Concentration')
plt.ylabel('Rate')
plt.legend(loc = 4)
plt.show()